In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/combined_ch_19.zip /content

In [ ]:
!unzip /content/combined_ch_19.zip -d /content

Streaming output truncated to the last 5000 lines.
  inflating: /content/combined_ch_19/schizophrenic/sub67_trial36_cond1_deit_1x19.png  
  inflating: /content/combined_ch_19/schizophrenic/sub67_trial36_cond2_deit_1x19.png  
  inflating: /content/combined_ch_19/schizophrenic/sub67_trial66_cond1_deit_1x19.png  
  inflating: /content/combined_ch_19/schizophrenic/sub67_trial66_cond2_deit_1x19.png  
  inflating: /content/combined_ch_19/schizophrenic/sub67_trial66_cond3_deit_1x19.png  
  inflating: /content/combined_ch_19/schizophrenic/sub67_trial67_cond1_deit_1x19.png  
  inflating: /content/combined_ch_19/schizophrenic/sub67_trial67_cond2_deit_1x19.png  
  inflating: /content/combined_ch_19/schizophrenic/sub67_trial67_cond3_deit_1x19.png  
  inflating: /content/combined_ch_19/schizophrenic/sub67_trial68_cond1_deit_1x19.png  
  inflating: /content/combined_ch_19/schizophrenic/sub67_trial68_cond2_deit_1x19.png  
  inflating: /content/combined_ch_19/schizophrenic/sub67_trial68_cond3_deit_1x1

Dataset

In [ ]:
from transformers import AutoImageProcessor, DeiTForImageClassification
import torch
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from torchvision import transforms

# Image preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load your dataset
dataset = ImageFolder(root='/content/combined_ch_19/', transform=transform)

generator1 = torch.Generator().manual_seed(42)

# Split into train and validation sets
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset,test_dataset = random_split(dataset, [train_size, val_size, test_size],generator=generator1)
# Data Loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

Training Loop

In [ ]:


# Load the image processor and model (DeiTForImageClassification for fine-tuning)
image_processor = AutoImageProcessor.from_pretrained("facebook/deit-base-distilled-patch16-224")
model = DeiTForImageClassification.from_pretrained("facebook/deit-base-distilled-patch16-224", num_labels=len(dataset.classes))

# Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)  # Example optimizer
loss_fn = torch.nn.CrossEntropyLoss()

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

num_epochs = 10

for epoch in range(num_epochs):
    # Training loop
    model.train()
    for batch in train_loader:
        images, labels = batch
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {loss.item()}")

    # Validation loop
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in val_loader:
            images, labels = batch
            images, labels = images.to(device), labels.to(device)

            outputs = model(images, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

            _, predicted = torch.max(outputs.logits, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss /= len(val_loader)
    accuracy = 100 * correct / total

    print(f"Epoch {epoch + 1}/{num_epochs}, Validation Loss: {val_loss:.4f}, Accuracy: {accuracy:.2f}%")


# Save the trained model
torch.save(model.state_dict(), 'trained_deit_model_kaggle_ch_combined.pth')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


config.json:   0%|          | 0.00/69.6k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/349M [00:00<?, ?B/s]

Some weights of DeiTForImageClassification were not initialized from the model checkpoint at facebook/deit-base-distilled-patch16-224 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/349M [00:00<?, ?B/s]

Epoch 1/10, Training Loss: 0.6381044983863831
Epoch 1/10, Validation Loss: 0.5371, Accuracy: 73.04%
Epoch 2/10, Training Loss: 0.17491906881332397
Epoch 2/10, Validation Loss: 0.3850, Accuracy: 81.71%
Epoch 3/10, Training Loss: 0.1365876942873001
Epoch 3/10, Validation Loss: 0.3312, Accuracy: 86.41%
Epoch 4/10, Training Loss: 0.3250976502895355
Epoch 4/10, Validation Loss: 0.3399, Accuracy: 87.27%
Epoch 5/10, Training Loss: 0.0973137691617012
Epoch 5/10, Validation Loss: 0.3722, Accuracy: 87.92%
Epoch 6/10, Training Loss: 0.017939865589141846
Epoch 6/10, Validation Loss: 0.4116, Accuracy: 87.92%
Epoch 7/10, Training Loss: 0.0024566894862800837
Epoch 7/10, Validation Loss: 0.4616, Accuracy: 87.83%
Epoch 8/10, Training Loss: 0.0007611545734107494
Epoch 8/10, Validation Loss: 0.5193, Accuracy: 87.53%
Epoch 9/10, Training Loss: 0.3385251760482788
Epoch 9/10, Validation Loss: 0.4937, Accuracy: 88.35%
Epoch 10/10, Training Loss: 0.001055905711837113
Epoch 10/10, Validation Loss: 0.5920, Accu

Download Trained Model

In [ ]:
# prompt: download the trained model

from google.colab import files

files.download('trained_deit_model_test_1_Kaggle.pth')


Load the Model for Testing

In [ ]:
model = DeiTForImageClassification.from_pretrained("facebook/deit-base-distilled-patch16-224", num_labels=len(dataset.classes))
model.load_state_dict(torch.load('/content/drive/MyDrive/trained_deit_model.pth'))

Some weights of DeiTForImageClassification were not initialized from the model checkpoint at facebook/deit-base-distilled-patch16-224 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [ ]:
# Testing dataset Loader
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Move the model to the device
model.to(device)

DeiTForImageClassification(
  (deit): DeiTModel(
    (embeddings): DeiTEmbeddings(
      (patch_embeddings): DeiTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): DeiTEncoder(
      (layer): ModuleList(
        (0-11): 12 x DeiTLayer(
          (attention): DeiTAttention(
            (attention): DeiTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
            )
            (output): DeiTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): DeiTIntermediate(
            (dense): Linear(in_features=768, out_features=3072, bias=True)
          

Testing Loop

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

test_loss = 0.0
correct = 0
total = 0
all_predicted = []
all_labels = []


with torch.no_grad():
    for batch in test_loader:
        images, labels = batch
        images, labels = images.to(device), labels.to(device)

        outputs = model(images, labels=labels)
        loss = outputs.loss
        test_loss += loss.item()

        _, predicted = torch.max(outputs.logits, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        all_predicted.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_loss /= len(test_loader)
accuracy = 100 * correct / total

# Calculate precision, recall, and F1 score
precision = precision_score(all_labels, all_predicted, average='binary')
recall = recall_score(all_labels, all_predicted, average='binary')
f1 = f1_score(all_labels, all_predicted, average='binary')

print(f"Test Loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}%")
print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

Test a random image

In [ ]:
image_path = '/content/data/schizophrenic/s01.edf0_11.png'  # Replace with the path to your image
image = Image.open(image_path).convert('RGB')

plt.imshow(image)
plt.axis('off')  # Turn off axis numbers and ticks
plt.show()

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

image = transform(image)
image = image.unsqueeze(0)  # Add a batch dimension

# Move the image to the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
image = image.to(device)
model.to(device)

# Make the prediction
with torch.no_grad():
    outputs = model(image)
    _, predicted_class = torch.max(outputs.logits, 1)

# Get the class name
class_names = dataset.classes  # Assuming you have 'dataset' from training
predicted_class_name = class_names[predicted_class.item()]

print(f"Predicted class: {predicted_class_name}")

Predicted class: schizophrenic
